In [1]:
# TARDIS imports for the gamma-ray code
from tardis.io.atom_data import AtomData
from tardis.model import SimulationState
from tardis.io.configuration import config_reader
from tardis.io.atom_data.util import download_atom_data
from tardis.energy_input.energy_source import get_nuclear_lines_database

from tardis.energy_input.gamma_ray_channel import evolve_mass_fraction


Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

In [2]:
# Download the atom data file from tardis-refdata repo to run this cell.
download_atom_data('kurucz_cd23_chianti_H_He')
atom_data_file = 'kurucz_cd23_chianti_H_He.h5'
atom_data = AtomData.from_hdf(atom_data_file)

INFO:tardis.io.atom_data.base:Reading Atom Data with: UUID = 6f7b09e887a311e7a06b246e96350010 MD5  = 864f1753714343c41f99cb065710cace 
INFO:tardis.io.atom_data.base:Non provided Atomic Data: synpp_refs, photoionization_data, yg_data, two_photon_data, linelist


In [3]:
# Read the config file and create a configuration object
config = config_reader.Configuration.from_yaml("tardis_configv1_density_exponential_nebular.yml")

In [ ]:
# Create the model
model = SimulationState.from_csvy(config, atom_data)


In [5]:
gamma_ray_lines = get_nuclear_lines_database(atom_data_file)

In [6]:
raw_isotope_abundance = model.composition.raw_isotope_abundance

In [7]:
evolve_mass_fraction(raw_isotope_abundance, 0.01, 20, 5, 0.00115741)

INFO:tardis.model.matter.decay:Decaying abundances for 864.0 seconds
INFO:tardis.model.matter.decay:Decaying abundances for 432647.99999999994 seconds
INFO:tardis.model.matter.decay:Decaying abundances for 864431.9999999999 seconds
INFO:tardis.model.matter.decay:Decaying abundances for 1296216.0 seconds
INFO:tardis.model.matter.decay:Decaying abundances for 1728000.0 seconds


0             1             2   \
time      atomic_number mass_number                                             
0.001157  28            56           2.017165e-19  3.558518e-19  4.814887e-19   
          26            52           4.092572e-16  5.694699e-16  6.407937e-16   
          24            48           4.921651e-13  5.577111e-13  5.603678e-13   
0.010000  22            48           8.215990e-19  9.310185e-19  9.354536e-19   
          23            48           3.781934e-15  4.285607e-15  4.306022e-15   
          24            48           4.883822e-13  5.534244e-13  5.560607e-13   
                        52           1.631387e-18  2.270030e-18  2.554342e-18   
          25            52           6.484354e-18  9.022795e-18  1.015287e-17   
          26            52           4.011119e-16  5.581359e-16  6.280402e-16   
                        56           1.032277e-26  1.821058e-26  2.464000e-26   
          27            56           2.300041e-22  4.057545e-22  5.490100e-22   
          28            56           2.014865e-19  3.554460e-19  4.809396e-19   
5.007500  22            48           7.312886e-14  8.286807e-14  8.326283e-14   
          23            48           4.086817e-13  4.631094e-13  4.653155e-13   
          24            48           1.033013e-14  1.170589e-14  1.176165e-14   
                        52           4.050666e-16  5.636388e-16  6.342323e-16   
          25            52           7.713721e-22  1.073342e-21  1.207774e-21   
          26            52           1.738035e-20  2.418427e-20  2.721326e-20   
                        56           2.125992e-21  3.750501e-21  5.074651e-21   
          27            56           8.566451e-20  1.511223e-19  2.044775e-19   
          28            56           1.139222e-19  2.009723e-19  2.719275e-19   
10.005000 22            48           1.543282e-13  1.748814e-13  1.757145e-13   
          23            48           3.375863e-13  3.825457e-13  3.843680e-13   
          24            48           2.185003e-16  2.475999e-16  2.487794e-16   
                        52           4.069838e-16  5.663065e-16  6.372342e-16   
          25            52           3.342389e-26  4.650839e-26  5.233339e-26   
          26            52           7.530982e-25  1.047915e-24  1.179162e-24   
                        56           7.079428e-21  1.248895e-20  1.689829e-20   
          27            56           1.302182e-19  2.297203e-19  3.108253e-19   
          28            56           6.441264e-20  1.136315e-19  1.537502e-19   
15.002500 22            48           2.201694e-13  2.494913e-13  2.506798e-13   
          23            48           2.719531e-13  3.081714e-13  3.096395e-13   
          24            48           4.621662e-18  5.237170e-18  5.262118e-18   
                        52           4.080061e-16  5.677289e-16  6.388347e-16   
          25            52           1.448271e-30  2.015229e-30  2.267628e-30   
          26            52           3.263207e-29  4.540661e-29  5.109361e-29   
                        56           1.346699e-20  2.375736e-20  3.214513e-20   
          27            56           1.518221e-19  2.678321e-19  3.623929e-19   
          28            56           3.641947e-20  6.424825e-20  8.693172e-20   
20.000000 22            48           2.731844e-13  3.095668e-13  3.110415e-13   
          23            48           2.189378e-13  2.480957e-13  2.492775e-13   
          24            48           9.775621e-20  1.107753e-19  1.113030e-19   
                        52           4.085562e-16  5.684944e-16  6.396961e-16   
          25            52           6.275423e-35  8.732074e-35  9.825732e-35   
          26            52           1.413962e-33  1.967488e-33  2.213908e-33   
                        56           2.050809e-20  3.617870e-20  4.895193e-20   
          27            56           1.606073e-19  2.833303e-19  3.833628e-19   
          28            56           2.059189e-20  3.632653e-20  4.915196e-20 